# Statistical Benchmarking Tutorial for Apache Mahout QDP

This notebook demonstrates how to use the new statistical benchmarking and visualization features added in Phases 1-3.

## Features Covered

1. **Phase 1**: Using benchmark utilities directly
2. **Phase 2**: Running benchmarks in statistical mode
3. **Phase 3**: Generating publication-ready visualizations
4. **Phase 4**: Best practices for reproducible benchmarks

## Prerequisites

- Python 3.9+
- CUDA-capable GPU (for GPU benchmarks)
- Mahout QDP installed with benchmark dependencies

## Setup

First, let's import the necessary modules and check our environment:

In [ ]:
import sys
import torch
import numpy as np
from pathlib import Path

# Check CUDA availability
print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

## Part 1: Using Benchmark Utils Directly

The benchmark_utils package provides low-level utilities for fair benchmarking.

In [ ]:
# Add benchmark directory to path
benchmark_dir = Path.cwd().parent if (Path.cwd().parent / 'benchmark_utils').exists() else Path.cwd()
sys.path.insert(0, str(benchmark_dir))

from benchmark_utils import (
    warmup,
    clear_all_caches,
    benchmark_with_cuda_events,
    compute_statistics,
    format_statistics,
    BenchmarkVisualizer,
)

print("✓ Benchmark utilities loaded successfully")

### Example 1: Basic GPU Operation Benchmarking

Let's benchmark a simple matrix multiplication operation with proper warmup and timing.

In [ ]:
def simple_matmul():
    """Simple matrix multiplication on GPU."""
    x = torch.randn(1000, 1000, device='cuda')
    return (x @ x.T).sum()

# Benchmark with CUDA events
print("Benchmarking matrix multiplication (1000x1000)...")
timings = benchmark_with_cuda_events(
    simple_matmul,
    warmup_iters=5,
    repeat=20
)

# Compute and display statistics
stats = compute_statistics(timings)
print("\nResults:")
print(format_statistics(stats))

### Example 2: Comparing Multiple Operations

Let's compare different matrix sizes to see scaling behavior.

In [ ]:
sizes = [512, 1024, 2048]
results = {}
results_raw = {}

for size in sizes:
    def matmul_op():
        x = torch.randn(size, size, device='cuda')
        return (x @ x.T).sum()
    
    print(f"\nBenchmarking {size}x{size} matrix...")
    timings = benchmark_with_cuda_events(matmul_op, warmup_iters=3, repeat=15)
    
    name = f"Size {size}x{size}"
    results[name] = compute_statistics(timings)
    results_raw[name] = timings
    
    print(f"  Mean: {results[name]['mean']:.2f} ms ± {results[name]['std']:.2f} ms")

print("\n" + "="*70)
print("Summary")
print("="*70)
for name in results:
    s = results[name]
    print(f"{name}: {s['mean']:.2f} ms (P95: {s['p95']:.2f} ms, CV: {s['cv']*100:.2f}%)")

### Example 3: Creating Visualizations

Now let's create publication-ready plots from our benchmark results.

In [ ]:
# Create visualizer
visualizer = BenchmarkVisualizer()

# Create output directory
output_dir = Path('./tutorial_results')
output_dir.mkdir(exist_ok=True)

# Generate all plots
visualizer.create_all_plots(
    results=results,
    results_raw=results_raw,
    output_dir=output_dir,
    prefix='matmul_comparison'
)

print(f"\nPlots saved to {output_dir}/")
print("Generated files:")
for f in sorted(output_dir.glob('matmul_comparison_*')):
    print(f"  - {f.name}")

### Display the generated plots

Let's view the visualizations we created:

In [ ]:
from IPython.display import Image, display, Markdown

# Display bar chart
print("Bar Chart with Error Bars:")
display(Image(filename=str(output_dir / 'matmul_comparison_bars.png')))

# Display box plot
print("\nBox Plot:")
display(Image(filename=str(output_dir / 'matmul_comparison_box.png')))

# Display violin plot
print("\nViolin Plot:")
display(Image(filename=str(output_dir / 'matmul_comparison_violin.png')))

# Display markdown table
print("\nStatistics Table:")
with open(output_dir / 'matmul_comparison_table.md', 'r') as f:
    display(Markdown(f.read()))

## Part 2: Running E2E Benchmark in Statistical Mode

The E2E benchmark measures end-to-end latency from disk to GPU. Let's run it with statistical mode.

In [ ]:
# Run E2E benchmark in statistical mode
# Note: This requires benchmark data files to be present

!cd .. && python benchmark_e2e.py \
    --statistical \
    --warmup 3 \
    --repeat 10 \
    --qubits 16 \
    --samples 100 \
    --frameworks mahout-parquet

## Part 3: Running with Visualization

Now let's run the same benchmark but generate publication-ready plots.

In [ ]:
# Run with visualization enabled
!cd .. && python benchmark_e2e.py \
    --statistical \
    --visualize \
    --warmup 3 \
    --repeat 10 \
    --qubits 16 \
    --samples 100 \
    --frameworks mahout-parquet \
    --output-dir ./tutorial_e2e_results

In [ ]:
# Display the generated E2E plots
e2e_dir = Path('../tutorial_e2e_results')
if e2e_dir.exists():
    print("Generated E2E visualization files:")
    for f in sorted(e2e_dir.glob('*.png')):
        print(f"\n{f.name}:")
        display(Image(filename=str(f)))

## Part 4: Best Practices for Reproducible Benchmarks

### 1. Always Use Warmup

Warmup iterations eliminate JIT compilation overhead and stabilize measurements.

In [ ]:
# BAD: No warmup - first run includes JIT overhead
def benchmark_no_warmup():
    timings = []
    for i in range(10):
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)
        start.record()
        simple_matmul()
        end.record()
        torch.cuda.synchronize()
        timings.append(start.elapsed_time(end))
    return timings

# GOOD: With warmup
timings_with_warmup = benchmark_with_cuda_events(
    simple_matmul,
    warmup_iters=5,
    repeat=10
)

print("Without proper warmup: First run may be 2-10x slower!")
print("Always use warmup for fair benchmarks.")

### 2. Clear Caches Between Frameworks

When comparing frameworks, clear caches to ensure fair comparison.

In [ ]:
# Benchmark Framework A
clear_all_caches()
timings_a = benchmark_with_cuda_events(simple_matmul, warmup_iters=3, repeat=10)

# IMPORTANT: Clear cache before next framework
clear_all_caches()

# Benchmark Framework B
timings_b = benchmark_with_cuda_events(simple_matmul, warmup_iters=3, repeat=10)

print("✓ Caches cleared between framework benchmarks")

### 3. Report Full Distributions, Not Just Means

Mean alone can be misleading. Always report percentiles and standard deviation.

In [ ]:
timings = benchmark_with_cuda_events(simple_matmul, warmup_iters=3, repeat=20)
stats = compute_statistics(timings)

# BAD: Only reporting mean
print(f"BAD:  Mean time: {stats['mean']:.2f} ms")

# GOOD: Reporting full distribution
print(f"\nGOOD: Mean: {stats['mean']:.2f} ms ± {stats['std']:.2f} ms")
print(f"      Median (P50): {stats['median']:.2f} ms")
print(f"      P95: {stats['p95']:.2f} ms, P99: {stats['p99']:.2f} ms")
print(f"      Range: [{stats['min']:.2f}, {stats['max']:.2f}] ms")
print(f"      CV: {stats['cv']*100:.2f}% (lower is better)")

### 4. Use Sufficient Repetitions

More repetitions give more reliable statistics. Aim for:
- Fast operations (< 1ms): 100+ repetitions
- Medium operations (1-100ms): 20-50 repetitions
- Slow operations (> 100ms): 10-20 repetitions

In [ ]:
# Compare different repetition counts
for n_repeat in [5, 10, 20, 50]:
    timings = benchmark_with_cuda_events(simple_matmul, warmup_iters=3, repeat=n_repeat)
    stats = compute_statistics(timings)
    print(f"N={n_repeat:3d}: Mean={stats['mean']:.2f}ms, Std={stats['std']:.2f}ms, CV={stats['cv']*100:.2f}%")

print("\nNotice: More repetitions → more stable statistics (lower CV)")

### 5. Save Configuration for Reproducibility

Always document your benchmark configuration.

In [ ]:
from benchmark_utils.config import BenchmarkConfig

# Create configuration
config = BenchmarkConfig.default()
config.fairness.warmup_iters = 5
config.fairness.repeat_runs = 20
config.visualization.output_dir = "./my_results"

# Save for reproducibility
config.to_yaml('./my_benchmark_config.yaml')

print("Configuration saved. Others can reproduce your benchmarks with:")
print("  config = BenchmarkConfig.from_yaml('my_benchmark_config.yaml')")

## Summary

This tutorial covered:

1. ✅ Using benchmark_utils for direct benchmarking
2. ✅ Creating publication-ready visualizations
3. ✅ Running statistical benchmarks from command line
4. ✅ Best practices for reproducible benchmarks:
   - Always use warmup
   - Clear caches between frameworks
   - Report full distributions
   - Use sufficient repetitions
   - Save configuration

## Next Steps

- Run full E2E benchmarks with `--statistical --visualize`
- Compare your quantum framework against baselines
- Use the generated plots in papers and blog posts
- Share your benchmark configurations for reproducibility

For more information, see:
- [Benchmark Roadmap RFC](../../docs/BENCHMARK_ROADMAP.md)
- [Benchmark Utils API Documentation](../benchmark_utils/README.md)
- [Main Benchmark README](../README.md)